In [1]:
"""
In this tutorial, we will show how to use KvikIO to read and write GPU memory directly to/from Zarr files.
"""
import shutil
import cupy
import zarr
import kvikio
import kvikio.zarr

We need to set three Zarr arguments: 
 - `meta_array`: in order to make Zarr read into GPU memory (instead of CPU memory), we set the `meta_array` argument to an empty CuPy array. 
 - `store`: we need to use a GPU compatible Zarr Store, which will be KvikIO’s GDS store in our case. 
 - `compressor`: finally, we need to use a GPU compatible compressor (or `None`). KvikIO provides a collection of nvCOMP compressor in `kvikio.zarr.nvcomp_compressors`.

In [2]:
kvikio.zarr.nvcomp_compressors

[kvikio.zarr.ANS,
 kvikio.zarr.Bitcomp,
 kvikio.zarr.Cascaded,
 kvikio.zarr.Gdeflate,
 kvikio.zarr.LZ4,
 kvikio.zarr.Snappy]

In [3]:
# Let's create a new Zarr array using KvikIO's GDS store and Snappy compression
z = zarr.array(
    cupy.arange(100), 
    chunks=10, 
    store=kvikio.zarr.GDSStore("my-zarr-file.zarr"), 
    meta_array=cupy.empty(()),
    compressor=kvikio.zarr.Snappy(),
    overwrite=True,
)
z

<zarr.core.Array (100,) int64>

In [4]:
# And because we set the `meta_array` argument, reading the Zarr array returns a CuPy array
type(z[:])

cupy.ndarray

From this point onwards, `z` can be used just like any other Zarr array.

In [5]:
z[1:9]

array([1, 2, 3, 4, 5, 6, 7, 8])

In [6]:
z[:] + 42

array([ 42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141])

In [7]:
# Clean up
shutil.rmtree("my-zarr-file.zarr", ignore_errors=True)